# Homework 1 - Slow general algorithms for sequence labeling

For homework 1, we will mainly be looking at a contrived task described below.  In this first homework, you will use slow but general algorithms.  They could be applied to *any* finite sequence problem, since they do not take advantage of any special problem structure.

The coding problems in this assignment are designed to be fairly small with each taking around **5-20 lines of code**.  Throughout this assignment you will see parameters `xx`, `yy`, `oo`, `aa`, which correspond to  $\mathbf{x},
\mathbf{y}, \mathbf{o}, \mathbf{a}$ from [the formalisms document](https://seq2class.github.io/scribe-notes/formalisms.pdf): the doubled letter is meant to suggest that the variable represents a string.  So `xx` is the input string, `yy` ranges over possible output strings, `oo` is a (partial) observation of the output string, and `aa` ranges over over possible decisions (predictions or other plans) that our system can choose.

## Task overview

For this assignment we are predicting which vowels in a word are stressed.
In the (made-up) natural language used in our dataset, every vowel sound is 
represented by one of the letters `aeiou`, and we will use `AEIOU`
to indicate their stressed versions.

Following the notation defined so far, the input sequence $\mathbf{x}$ is our input
word (with no capitalization).  Each possible output $\mathbf{y}$ 
contains the same sequence of letters as $\mathbf{x}$, but with some of them 
`cApitalIzed` to indicate stress.  Therefore, if $\mathbf{x}$ contains $m$ vowels, 
then $|\mathcal{Y}_{\mathbf{x}}| = 2^m$.  Sometimes, however, this exponentially
large space gets reduced because for *some* vowels, we observe whether they are 
stressed or unstressed.  The observation is a string $\mathbf{o}$ that is 
a version of $\mathbf{y}$ with some of the vowels already given correctly 
as capital or lowercase, but unobserved vowels replaced with `?`.  So if 
$\mathbf{o}$ contains $n$ question marks (where $0 \leq n \leq m$), 
then $|\mathcal{Y}_{\mathbf{x,o}}| = 2^n$.


### Sample inputs and outputs
| $\mathbf{x}$ | $\mathbf{o}$ | $\mathbf{y} \in \mathcal{Y}_{\mathbf{x},\mathbf{o}}$ | $\mathbf{y} \notin \mathcal{Y}_{\mathbf{x},\mathbf{o}}$ (illegal)|
|:-------------:|:------------:|:---------------------:|:--------------------|
| `helilela` | `h?lil?l?` | `helilela` | `Helilela` |
| | | `helilelA` | `helIlela` |
| | | `hElilelA` | ... |
| | |    ...     |     |
| `idonotul` | `?don?t?l` | `IdonotUl` | `idOnotUl` |
| | | ... | ... |

## Assignment Instructions

Follow through the assignment in order.  Each text cell will provide details about the task and explain what is expected.  Inside of code blocks, you will see `### STUDENTS START` to indicate where you are expected to edit the cell and fill in some code.  There are also a few short-answer questions throughout the assignment.  These are marked in red with <span style='color:red'>FILL IN</span>; again, edit the cell to give your answer.

In [ ]:
import numpy as np
import scipy.optimize
from collections import namedtuple
import csv
import sys
import re

Data_type = namedtuple('Data', ['xx', 'oo', 'yy'])

def iterate_data(filename='train', *, max_examples=None):
    file = open(filename+'.tsv')
    for n, row in enumerate(csv.DictReader(file, delimiter='\t')):
        if max_examples and n >= max_examples:
            break
        yield Data_type(
            xx=tuple(row['xx']),
            oo=tuple(row['oo']) if 'oo' in row else None,
            yy=tuple(row['yy']) if 'yy' in row else None,
        )

## Let's start by looking at the data ...

Our *strings* are sequences of symbols from our alphabet, represented as Python tuples.  Although our symbols in this problem are letters, they might be words in future problems, which is why we don't use raw Python strings.

Each example consists of a triple `(xx,oo,yy)` where the observation string `oo` may include the special symbol `?`.
* A **training example** may specify either 
  * `oo=None` and `yy` is the fully observed output
  * `yy=None` and `oo` is a partial observation of the output (see formalisms.pdf, "Observations")

In [ ]:
itr = iterate_data()   # uses training set by default
print(next(itr))       # look at the first example in our training set
next(itr)              # look at the second example in our training set

* A **test example** consists of a triple `(xx,oo,yy)`, where `yy` is the correct answer.
  The prediction rule will be given `(xx,oo)` and asked to predict `yy`.
  * Usually `oo=None`, so the rule must simply predict `yy` from `xx`.  
  * However, if `oo` is specified, this informs the predictor that the true `yy` is compatible with `oo`, 
    which should improve the prediction.

In [ ]:
itr = iterate_data('dev')   # okay to peek at the development dataset (but not the real test data!)
print(next(itr))            # oo as partially observed output
next(itr)

## Setting up the task

Here is the base interface that we will be extending for the remainder of this homework.
All of our methods are defined with a `*` argument, like this:
```python
def mymethod(self, *, xx, oo=None, yy):
    pass   
```
The `,*,` prevents the parameters `xx, oo, yy` from being passed as positional arguments; instead they must be passed as keyword arguments.  This makes the code easier to read, and prevents errors due to accidentally specifying the arguments in the wrong order.


`TaskSetting` defines the environment in which we are operating, methods such as `iterate_y` and `iterate_yy` defines the output $\mathcal{Y}$.  The method `reward` defines the the environment's reward function $R(\mathbf{a} \mid \mathbf{x}, \mathbf{y})$ that returns how good our prediction $a$ was given the true answer is $y$.  We will start implementing the methods on this class in the [first problem](#Problem_stresstask).
<a id="TaskSetting"></a>

In [ ]:
class TaskSetting(object):
    """
    Base class for our task.
    Defines the domain of YY, and our actions AA.  
    It also defines the reward function that we are trying to maximzie 
    """
    
    def iterate_yy(self, *, xx, oo=None):
        """
        Returns an iterator over legal `yy` sequences (represented as tuples).  
        If an observation `oo` is specified, restricts to `yy` sequences that 
        are consistent with `oo`.
        This method *defines* the space of output strings that we will consider 
        (although some of those could turn out to have probability 0).
        Usually it is implemented by calling `self.iterate_y`.
        """
        raise NotImplementedError()     
        
    def iterate_y(self, *, xx, oo=None, yy_prefix):
        """
        Returns an iterator over legal next characters of `yy`.
        In other words, returns all characters `y` such that
        the concatenation `y_prefix + y` is a prefix of some
        output `yy` that is legal given input string `xx` 
        and observable `oo`. 
        (This set is discussed in formalisms.pdf, "Restricting summations to the output space".)
        """
        raise NotImplementedError()
        
    def iterate_aa(self, *, xx):
        """
        Returns an iterator over plans that are allowed for input `xx`.
        The default implementation just calls `iterate_yy(xx=xx)`, which is 
        appropriate for prediction tasks where the plans simply correspond 
        to predicting the different outputs.  This can be overridden for
        other kinds of decision tasks.
        (See formalisms.pdf, "Decision theory" and "More decision theory".)
        """
        yield from self.iterate_yy(xx=xx)
        
    def reward(self, *, aa, xx, yy):
        """
        Return the reward that plan `aa` will get on input `xx` if the true answer is `yy`.
        This method *defines* the reward function.
        """
        raise NotImplementedError()

`ProbabilityModel` defines $P(\mathbf{y} \mid \mathbf{x})$.  It does this by defining a `score` function which corresponds to $G(\mathbf{x}, \mathbf{y})$ as well as functions to train and sample from our distribution.  We have given you a partial implementation of this class [below](#Problem_linearstressmodel) which you will fill in.
Because our probability model must compute probabilities $P(\mathbf{y} \mid \mathbf{x})$, it needs to be able to enumerate the domain of $\mathbf{y} \in \mathcal{Y}$, and thus has the task model defined above as `self.task`.
<a id="ProbabilityModel"></a>

In [ ]:
class ProbabilityModel(object):
    """
    Base class for our probability model P_\theta(y | x, o).
    """
    
    def __init__(self, task):
        assert isinstance(task, TaskSetting)
        self.task = task
        self.initialize_params()
    
    def initialize_params(self):
        """
        Reset the model parameters to their start state.
        """
        # by default there are no parameters to initialize
        pass
    
    @property
    def params(self):
        """
        Return a copy of the parameter vector for this model
        """
        raise NotImplementedError()
    
    @params.setter
    def params(self, params_vec):
        """
        Update the parameters for the model given vector params_vec
        """
        raise NotImplementedError()
        
    def score(self, *, yy, xx):
        """
        Return the score G(`yy`, `xx`) with respect to the params.
        By default use `score_with_gradient` and only return the score
        """
        score, gradient = self.score_with_gradient(yy=yy, xx=xx)
        return score
        
    def score_with_gradient(self, *, yy, xx):
        """
        Return two values: the score G(`yy`,`xx`) and its gradient with respect to the params.
        It's convenient to compute the gradient along with the score, and we'll need it later.
        """
        raise NotImplementedError()
  
    def normalizer(self, *, xx, oo=None):
        """
        The normalizing function `Z(xx)` or `Z(xx,oo)`, 
        often called the "partition function", that is used to define
            p(yy | xx)     = \frac{1}{Z(xx)}    exp G(...)
            p(yy | xx, oo) = \frac{1}{Z(xx,oo)} exp G(...)
        
        The default implementation computes this by a brute-force sum with `iterate_yy`,
        but that could be overridden by a more efficient method when available.
        (See formalisms.pdf, "Marginal and conditional probabilities".)
        """
        raise NotImplementedError()

    def prob(self, *, xx, oo=None, yy=None):
        """
        Return p(yy | xx) or p(oo | xx).  Only one of `yy` or `oo` should be specified.
        Computed using the scoring model G (the `model` attribute).
        
        If `yy` is not a legal string in the output space or `oo` is not a legal observable,
        we would ideally raise an error, but you are not required to implement that.
        """
        assert (oo is None) != (yy is None)
        raise NotImplementedError()
        
    def train_example(self, *, xx, oo=None, yy=None):  
        """
        Improve our model parameters on a single example.
        Either `yy` as a fully observed output or `oo` as a partial observation
        should be specified, but not both. (See formalisms.pdf, "Observations".)
        """
        raise NotImplementedError()
    
    def sample(self, *, xx, oo=None):
        """
        Return a sample `yy` from p(yy | xx) or p(yy | xx,oo)
        There is a generic way to do this, 
        """
        raise NotImplementedError()
    
    def train_epoch(self, dataset):
        """
        Train for one epoch.
        This means that we iterate once through `dataset`, passing 
        each example to `self.train_example`.
        """
        count = 0
        for xx, oo, yy in dataset:
            self.train_example(xx=xx, oo=oo, yy=yy)
            count += 1
            if count % 50 == 0: sys.stdout.write('\rtrained on {} examples'.format(count))  # print progress
    

`DecisionAgent` controls *how* we make predictions once we have both our `TaskSetting` as `self.task` and `ProbabilityModel` as `self.model`.  We will experiment with a few different implementations of our decision rule below.
<a id="DecisionAgent"></a>

In [ ]:
class DecisionAgent(object):
    """
    Base class for the decision agents in this homework.
    
    To define a particular task setting, create a subclass
    that specifies the space of outputs (`iterate_yy`),
    the space of plans (`iterate_aa`, which defaults to
    the space of outputs), and the reward function (`reward`).
    
    An instance of the subclass needs to also specify a particular 
    probability model (a `model` object) and a particular decision 
    rule (a `decision` function), which are passed as arguments 
    to the constructor.
    """

    def __init__(self, task, model):
        """Arguments to the constructor are a """
        super().__init__()
        assert isinstance(task, TaskSetting)
        assert isinstance(model, ProbabilityModel)
        self.model = model
        self.task = task
    
    def decision(self, *, xx, oo=None):
        """
        Return some action `aa` that is appropriate to input `xx` and the partially
        observed output `oo` (if any).  
        
        This might invoke `model`, `reward`, `iterate_aa`, and perhaps a random 
        number generator.
        
        This method will be replaced by the constructor.  It is included here
        to document the calling convention, and also because a subclass might
        define this method directly and override the constructor.
        """
        raise NotImplementedError()
            
    def test(self, dataset):
        """
        Run the predictor over `dataset` and return the average reward.
        """
        reward = 0
        count = 0
        for xx, oo, yy in dataset:
            aa = self.decision(xx=xx, oo=oo)
            reward += self.task.reward(aa=aa, xx=xx, yy=yy)
            count += 1
            if count % 50 == 0: sys.stdout.write('\revaluated {} examples'.format(count))
        return reward / count
    
    def show_errors(self, dataset, count=20, reward_less_than=.75):
        """
        Print out (up to) `count` examples in which the predictor got the 
        wrong answer.
        """
        for xx, oo, yy in dataset:
            aa = self.decision(xx=xx, oo=oo)
            if self.task.reward(aa=aa, xx=xx, yy=yy) < reward_less_than:
                print("yy: {yy}\n\taa: {aa}\n\txx: {xx}\n\too: {oo}\n".format(
                    xx=''.join(xx),
                    oo=''.join(oo),
                    yy=''.join(yy),
                    aa=''.join(aa)
                ))
                count -= 1
                if count == 0: return

<a id="Problem_stresstask"></a>
Here we extend our [`TaskSetting`](#TaskSetting) class to define our environment of stressing vowels:  
  * First fill in the function `iterate_y` to iterate through the domain of $\mathbf{y}_t$ conditioned on the input $\mathbf{x}, \mathbf{o}$ and the prefix of the y string $\mathbf{y}_{0:t-1}$ 
  * Then fill in `iterate_yy` to use `iterate_y` to iterate through the entire domain of $\mathcal{Y}_{\mathbf{x}, \mathbf{o}}$.  Sometimes the parameter `oo` will be `None` indicating that we instead want to iterate the domain of $\mathcal{Y}_\mathbf{x}$
  
You can define an iterator in python using `yield` as follows:
```python
def f():
    yield 'a'
    yield 'e'
for vowel in f():
    print(vowel)
```

In [ ]:
class StressTask(TaskSetting):
    """
    Class of models for the vowel stress problem, with
    a simple 0-1 reward function.
    """
    
    def reward(self, *, aa, xx, yy):
        return 1 if yy == aa else 0    # was the answer exactly right?
        
    def iterate_y(self, *, xx, oo=None, yy_prefix):
        """
        Iterate through the domain of yy_t given xx, oo, yy_{0:t-1}.
        """
        t = len(yy_prefix)
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
   
    def iterate_yy(self, *, xx, oo=None):
        """
        Iterate through Y using self.iterate_y
        """
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    
task = StressTask()

Now, using that class, let's check that your `iterate_yy` correctly iterates over possible output strings given $\mathbf{x}$ and optionally $\mathbf{o}$.  We give you two test cases here to get started.  You should add your own test cases as well to this notebook cell.  

(*Note*: Python uses `t = tuple(s)` and `s = ''.join(t)` to convert between a string `s` and a tuple `t` of its characters.)

In [ ]:
assert set(task.iterate_yy(xx=tuple('test'))) == {tuple('test'), tuple('tEst')}
[''.join(yy) for yy in task.iterate_yy(xx=tuple('testphrase'), oo=tuple('t?stphrAs?'))]
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


You'll now be able to create specific agents like this:
```python
mymodel = StressModel(task)
myagent = StressAgent(task, model)
```
and then you can train that agent on data (by training its model object) and ask it for probabilities and decisions.  
But wait - we don't yet have a `model` or a `decision` function!

<a id="Problem_linearstressmodel"></a>
## Building a model for our agent

For the first part of this assignment we will be using a simple linear scoring function $G_\theta(\mathbf{x}, \mathbf{y})$ with hand written features.

Our class `LinearStressModel` will define a linear scoring model $G$ for this task, with parameters $\theta$.  
Its `score` method can return a score $G_\theta(\mathbf{x},\mathbf{y}) \in \mathbb{R} \cup \{ -\infty \}$.
```python
from math import inf 
model = LinearStressModel(task)
assert model.score(xx=xx, yy=yy) == -inf
```

An instance of the class specifies a particular value for the parameters.  Its parameter vector $\theta$ can be examined via the `params` property, and should be updated like this:
```python
update = np.zeros_like(model.params)  # a zero vector with the same dimensionality as params
update[7] = 123                       # put some nonzero elements into that vector
update[8] = 456
model.params += update                # add it to params (+= makes Python invoke set_params)
```
This may seem a bit roundabout: why not just modify `params` directly via statements like `model.params[7] += 123`?  The reason is that the object does not actually have a `params` attribute.  Its parameters are stored in one or more attributes with other names.  Calling `params` invokes a "getter" function (the `params` property) that constructs a new vector of all those parameters.  Modifying that new vector would not change the original parameters, whereas calling `params += ...` invokes a "setter" function that does so.

Note that `LinearStressModel` is a *particular* linear scoring model, but it could be subclassed to include more features other models.

In [ ]:
class LinearStressModel(ProbabilityModel):   # would have been nicer to inherit from a Model base class

    def __init__(self, task):
        super().__init__(task)
        self.initialize_params()

    def initialize_params(self):
        np.random.seed(42)
        self._theta = np.array([
            .1,
            .05,
            0,
            1,
            0,
        ])
        
    @property
    def params(self):
        return np.array(self._theta)

    @params.setter
    def params(self, val):
        assert np.isfinite(val).all()
        self._theta[:] = val

    def features(self, *, xx, yy):
        """Extract a feature vector that measures various features of the pair (xx,yy)."""

        string = ''.join(yy)                          # concatenate symbols into an ordinary python string
        vowels = re.sub(r'[^aeiouAEIOU]','', string)  # extract just the vowels
        
        # All of our features are counts of structures in `(xx,yy)`.
        # For this problem, the features don't need to look at `xx`, but features for a POS tagger would do that.
        uppercase_vowels = len(re.findall(r'[AEIOU]', vowels))
        altcase_vowels = len(re.findall(r'(?=([aeiou][AEIOU])|([AEIOU][aeiou]))', vowels))
        enduppercase_vowels = len(re.findall(r'[AEIOU]$', vowels))
        uppercase_consonants = len(re.findall(r'[TNSRHDLC]', string))  
        length = len(string)

        # Assemble those counts into a feature vector.
        return np.array([uppercase_vowels,
                         altcase_vowels,
                         enduppercase_vowels,
                         uppercase_consonants,
                         length])

    def score_with_gradient(self, *, xx, yy):
        """
        Return two values: the score G(`xx`,`yy`) and its gradient with respect to the params.
        It's convenient to compute the gradient along with the score, and we'll need it later.
        """
        # The score is the dot product of params theta with the feature vector,
        # which implies that its gradient is just the feature vector.
        f = self.features(xx=xx, yy=yy)
        score = self._theta.dot(f)    
        return float(score), f
    
    def normalizer(self, *, xx, oo=None):
        """The normalizing function Z(xx) or Z(xx, oo)"""
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def prob(self, *, xx, oo=None, yy=None):
        """The probability: P(y | x) or P(o | x)"""
        assert (oo is None) != (yy is None)
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    
linear_model = LinearStressModel(task)

Let's try the model out with its default initial parameter setting.  
What is each feature doing in the cell below?

**Answer:** <span style='color:red'>FILL IN</span>

Why don't the fourth and fifth features help to discriminate among the `yy` candidates?  Will that be true for every `xx`, or just for `testphrase`?

**Answer:** <span style='color:red'>FILL IN</span>

To understand the features, it will probably help to experiment, so you may want to add your own examples to the cell below.

In [ ]:
testphrase = tuple('testphrase')
[( ''.join(yy),                                   # string yy
   linear_model.features(xx=testphrase, yy=yy),   # yy's feature vector with this xx
   linear_model.score(xx=testphrase, yy=yy)  )    # yy's score with this xx, using the current parameters 
 for yy in task.iterate_yy(xx=testphrase)]

Now go back to the [cell above](#Problem_linearstressmodel) and fill in the `normalizer` and `prob` methods.

Let's check your implementation.  
Using the scores above, give simple numerical expressions (which may refer to $Z$) for:

$p(\mathbf{Y}=\texttt{testphrAsE} \mid \mathbf{X}=\texttt{testphrase}) =$ <b><span style='color:red'>FILL IN</span></b>

$p(\mathbf{Y} \in \texttt{testphrAs?} \mid \mathbf{X}=\texttt{testphrase}) =$ <b><span style='color:red'>FILL IN</span></b>

where $\texttt{testphrAs?}$ is being informally used to mean the *set* of output strings compatible with that partial observation.  
Those numerical expressions should match the answers computed below. 

In [ ]:
Z = linear_model.normalizer(xx=testphrase)
from math import isclose
assert isclose(Z, 9.812839692144227)  # checks that you got the right value

[Z, 
 linear_model.prob(xx='testphrase',yy='testphrAsE'),
 linear_model.prob(xx='testphrase',oo='testphrAsE'),  # should be the same as previous line
 linear_model.prob(xx='testphrase',oo='testphrAs?')]  # should be bigger than previous line

Our agent needs a decision rule.  Define a "Viterbi" decision rule that chooses the *most probable* `yy` (which is also the one that scores highest):
$$\text{argmax}_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x}}} G_\theta(x, y)$$

If `oo` is specified, the rule should choose the most probable `yy` that is *compatible* with `oo`:
$$\text{argmax}_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x},\mathbf{o}}} G_\theta(x, y)$$

Our DecisionAgent defines `self.task` with all methods defined [above](#DecisionAgent) such as `self.task.iterate_yy` and our LinearStressModel with `self.model` 

In [ ]:
class ViterbiAgent(DecisionAgent):

    def decision(self, *, xx, oo=None):
        """viterbi decision rule"""
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
viterbi_agent = ViterbiAgent(task, linear_model)

Check that `viterbi_agent` is behaving as you expect, by trying out some more test cases in the following cell.

In [ ]:
viterbi_agent.decision(xx='testphrase')
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


Let's see what kind of predictions the agent makes on *development* data.  
(We haven't trained its parameters yet.)

Fill in assignments to `aa` (the prediction) and `reward` (the reward received as a result), so that the code prints out tuples $(R, \mathbf{x}, \mathbf{o}, \mathbf{y}, \mathbf{a})$.

In [ ]:
for xx, oo, yy in iterate_data('dev', max_examples=5):
    aa = reward = None  # replace with actual computation below
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    print((reward,
           ''.join(xx),
           ''.join(oo) if oo is not None else None,
           ''.join(yy) if yy is not None else None,
           ''.join(aa)))
    

Let's use the development dataset to check the overall quality of our predictions.
This returns the *average reward* over all development examples (so higher values are better).
Since we defined reward to be 1 if `aa==yy` and 0 otherwise, our average reward is the fraction of examples where the agent predicted `yy` exactly.

In [ ]:
viterbi_agent.test(iterate_data('dev'))

Some of our "success" probably came from the fact that the dev data included partial observations $\mathbf{o}$ of the output $\mathbf{y}$, so the agent only had to pick the output from the set $\boldsymbol{\mathcal{Y}}_{\mathbf{x}, \mathbf{o}}$ of outputs that are compatible with $\mathbf{o}$.

How much worse would we have done without $\mathbf{o}$, so that we have to pick the right output from the larger  set $\boldsymbol{\mathcal{Y}}_{\mathbf{x}}$?

In [ ]:
# iterate over versions of the examples for which oo is replaced by None
viterbi_agent.test([(xx,None,yy) for (xx,oo,yy) in iterate_data('dev')])

## Training the model

We will define our training objective as $A_{y \mid x}(\boldsymbol\theta) = \sum_{n=1}^N \log p_\boldsymbol\theta(\mathbf{y}_n \mid \mathbf{x}_n) - c || \boldsymbol\theta ||^2$ where we will maximize the log probability of our model and regularize our parameter vector $\theta$ using L2.  We will take a one step stochastic gradient descent per each sample in our training set.

We can also train the case that we have a partial observation $\mathbf{o}_n, \mathbf{x}_n$.  In this case we will be maximizing the log probability of the expression $P(\mathbf{o}_n \mid \mathbf{x}_n)$ using $P(\mathbf{o} \mid \mathbf{x}) = \sum_{\mathbf{y} \in \mathcal{Y}_{x,o}} P(\mathbf{y} \mid \mathbf{x})$.


To compute the update for this rule, we need to derive the gradient for $\nabla_\theta A(\theta)$.

Here is the computation for $\nabla_\theta A_{y \mid x}(\theta)$, you will have to derive for the $\nabla_\theta A_{o \mid x}(\theta)$ below.

$$
\begin{align*}
A_{y \mid x}(\boldsymbol\theta) &= \sum_{n=1}^N \log p_\boldsymbol\theta(\mathbf{y}_n | \mathbf{x}_n) - c || \boldsymbol\theta||^2 \\
&= \log \frac{\exp({G_\boldsymbol\theta(\mathbf{y}_n | \mathbf{x}_n)})}{Z_\theta(x)} - c || \boldsymbol\theta||^2 \\
&= G_\boldsymbol\theta(\mathbf{y}_n | \mathbf{x}_n) - \log Z_\theta(x) - ||\boldsymbol\theta|| ^2
\end{align*}
$$
Substituting $Z_\boldsymbol\theta(x) = \sum_{y \in Y_x} \exp({G_\boldsymbol\theta(\mathbf y_n \mid \mathbf x_n)})$, we get:
$$
\begin{align*}
&= G_\boldsymbol\theta(\mathbf{y}_n | \mathbf{x}) - \log \sum_{y \in Y} \exp({G_\boldsymbol\theta(\mathbf y_n | \mathbf x_n)}) - ||\boldsymbol\theta|| ^2 \\
\nabla_\theta A_y(\boldsymbol\theta) &= \nabla_\boldsymbol\theta G(\mathbf x \mid \mathbf y) - \frac{1}{Z_\boldsymbol\theta(x)} \sum_{\mathbf{y} \in \mathbf{Y}} \exp(G_\boldsymbol\theta(\mathbf y_n \mid \mathbf x_n)) \nabla_\boldsymbol\theta G_\boldsymbol\theta(\mathbf y_n \mid \mathbf x_n) - \frac{c}{2} * \boldsymbol \theta
\end{align*}
$$

Write out the expression for $A_{o \mid x}(\boldsymbol\theta)$.  Hint: $\mathbf{o}$ should **not** appear in the denominator or *free* expression.  You may find Jason's [tutorial](https://www.cs.jhu.edu/~jason/tutorials/loglin/formulas.pdf) on log linear models helpful

$$
\begin{align*}
A_{o \mid x}(\boldsymbol\theta) &= \color{red}{\text{FILL IN}} \\
\nabla_\theta A_{o \mid x}(\boldsymbol\theta) &= \color{red}{\text{FILL IN}}
\end{align*}
$$

To *maximize* the objective function using stochastic gradient accent our update rule is
$$\boldsymbol\theta \mathrel{+}= \alpha * \nabla_\boldsymbol\theta A(\boldsymbol\theta)$$
Where $\boldsymbol\theta$ is our parameter vector (`self.params`), $\nabla_\boldsymbol\theta A(\boldsymbol\theta)$ is a vector of gradients that we computed above and $\alpha$ is a scalar representing our learning rate.

In the above equations, $G_\theta$ is still the scalar from our score function and $\nabla_\boldsymbol\theta G$ refers to the gradient vector return by `score_with_gradient` on our [ProbabilityModel](#ProbabilityModel).  (`G, gradient_G = score_with_gradient(...)`)

The parameters of the score function can be updated with `self.params += update_vector` as discussed earlier.

In [ ]:
class TrainableLinearStressModel(LinearStressModel):

    # Use this for your learning rate
    alpha = .05
    
    # Use this for your L2 regularization
    c = .001
    
    def compute_gradient(self, *, xx, yy):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    
    def train_example(self, *, xx, oo=None, yy=None):
        assert (yy is None) != (oo is None)  # check that we have yy or oo
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    

trainable_linear_model = TrainableLinearStressModel(task)

Train the log-linear model.

In [ ]:
# pre training reward
trainable_linear_model.initialize_params()
viterbi_agent = ViterbiAgent(task, trainable_linear_model)
viterbi_agent.test(iterate_data('dev'))

In [ ]:
for i in range(2):
    trainable_linear_model.train_epoch(iterate_data())

Check how well the model works now.

In [ ]:
viterbi_agent.test(iterate_data('dev'))

Check the parameters that the model learns, which features does the model learn are the most important for doing well on this task?

<b><span style='color:red'>FILL IN</span></b>

In [ ]:
viterbi_agent.model.params

## A new reward function

So far we have been using a reward function which only rewards us when we are *exactly* right, however there are some scenarios where we might be able to receive a fractional reward in the case that we have most of the output correct.

Below we define a simple reward function which compares the hamming distance between two strings.
This function also defines an asymmetric reward in the case that we predict incorrectly. It is better to predict a lowercase letter rather than a capital one as we can receive a negative reward of -.3 for any lower case vowel rather than -1.

In [ ]:
class HammingTask(StressTask):
    
    def reward(self, *, aa, xx, yy):
        return sum(0 if aa[t] == yy[t] else (-.3 if aa[t] in 'aeiou' else -1.0) for t in range(len(aa))) / len(aa)

hamming_task = HammingTask()

In [ ]:
hamming_task.reward(aa='tEst', xx='test', yy='test')

In [ ]:
hamming_task.reward(aa='test', xx='test', yy='tEst')

## Reward-infused decoding

Our old decision rule -- the Viterbi decoder -- tries to maximize the old reward (0 or 1).
Let's change the decision rule to try to optimize its prediction for the new (arbitrary) reward function instead.
This is minimum Bayes risk (MBR) decoding, although since we've formulated the problem as maximizing expected reward rather than minimizing expected loss, we should really call it maximum Bayes value.
 $$ \mathrm{argmax}_{\mathbf{a} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x}}} \sum_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x},\mathbf{o}}} P(\mathbf{y} \mid \mathbf{x}, \mathbf{o}) \cdot R(\mathbf{a} \mid \mathbf{x},\mathbf{y}) $$

where $p(\mathbf{y} \mid \mathbf{x}, \mathbf{o}) = \frac{1}{Z(\mathbf{x}, \mathbf{o})} \exp G_\boldsymbol\theta(\mathbf{x}, \mathbf{y})$.  

In [ ]:
class MaximumBayesValueAgent(DecisionAgent):
    
    def decision(self, *, xx, oo):
        ### STUDENTS START
        ### Bayes min risk decoding
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    
max_bayes_value = MaximumBayesValueAgent(hamming_task, trainable_linear_model)

Test how well Bayes min risk performs with our Hamming distance reward.

Note: We are able to use the `trainable_linear_model` from before *without* retraining the model.  Our learned function defines $G_\boldsymbol\theta$ which defines our probability model over possible true underlying states $p(y \mid x, o)$ but we are trying to maximize our reward given our belief about which state $\mathbf{y}$ we are in.

In [ ]:
max_bayes_value.test(iterate_data('dev'))

## *"Crazy"* reward function
  
Our environment can define any reward function it wants which compares our generated value $\mathbf{a}$ and the known gold answer $\mathbf{y}$. One of the most common reward functions used in Natural Language Processing is BLEU.  BLEU is primarily used in machine translation. It compares the number of overlapping n-grams between the generated sequence $\mathbf{a}$ and the gold sequence $\mathbf{y}$.  Additionally, it contains a brevity penalty and can work when $\mathbf a$ and $\mathbf y$ are different lengths. This reward function has become popular for scoring machine translation systems as it has been shown to correlate with human preferences for translations, can be easily computed and it allows us to compare between two translations when neither is exactly correct.

In this homework, we will be implementing a slightly simplified version of BLEU in that for smoothing we are just going to use $\epsilon$ as defined below.  BLEU operates by computing the harmonic mean of the precision between n-grams of length $\{1,2,3,4\}$.  By smoothing our objective we will still recover information about our shorter n-grams even in the case that we fail to match any of the longer n-grams.

$$
\begin{align*}
  \#x_c &:= \text{Number of instances of ngram $c$ inside string $x$} \\
  \text{ngram}_n &:= \text{set of ngrams $n$ tokens long} \\
  \epsilon &:= .01 \\
  p_n &= \frac{\max\{\sum_{c \in \text{ngram}_n} \min\{\#\text{pred}_{c}, \#\text{gold}_{c}\}, \epsilon \} }{\max\left\{\sum_{c \in \text{ngram}_n} \#\text{gold}_c, 1\right\}} & \textit{Precision with ``smoothing''} \\
  \textit{bp}(c, r) &= \left\{ \begin{array}{cc} 1 & c \ge r \\ e^{1 - r/c} & \text{otherwise} \end{array} \right. & \textit{brevity penalty} \\
  \textit{bleu}(\text{pred}, \text{gold}) &:= \text{exp}\left( \frac{1}{4} \sum_{i\in[1, 4]} \log(p_i) \right) * \textit{bp}(\text{len}(\text{pred}), \text{len}(\text{gold}))
\end{align*}
$$

In [ ]:
### STUDENTS START
### Helper functions for bleu
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

def bleu(*, aa, yy):
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    

class BleuStressTask(StressTask):
    
    def reward(self, *, aa, xx, yy):
        return bleu(aa=aa, yy=yy)
    
bleu_task = BleuStressTask()

Sanity check our BLEU function

In [ ]:
assert bleu(aa='test', yy='test') == 1
assert np.isclose(bleu(aa='TEST', yy='test'), .0045180100180492264)

### STUDENTS START
### Write additional tests for your bleu function
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


Check how we are doing with BLEU

In [ ]:
bleu_max_bayes_value = MaximumBayesValueAgent(bleu_task, trainable_linear_model)

bleu_max_bayes_value.test(iterate_data('dev', max_examples=100))

## Speeding up the BLEU Implementation

It now appears that our test method has become somewhat slow after introducing the BLEU function.  When building more complicated models, we will often have to put our engineering hats on and figure out how to make our programs run faster.

First lets get a baseline for how fast our BLEU decoder is:

In [ ]:
%time bleu_max_bayes_value.test(iterate_data('dev', max_examples=100))

Let's try and identify the slow functions using [`%prun`](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-prun).  This will generate a profile of all the functions that are called when running and how long each function takes to run.

In [ ]:
%prun bleu_max_bayes_value.test(iterate_data('dev', max_examples=100))

Now that you have identified which functions are *slow* we are going to use [Cython](https://cython.readthedocs.io/en/latest/) to make our program faster by compiling the slow functions.

Cython uses python like syntax but adds a few additional annotations which allows cython to better compile a faster function.
     
#### Short Cython tutorial / hints
 1. In our jupyter notebook, we can load cython using `%load_ext cython`  we can then use cython inside of any jupyter cell by putting `%%cython` at the top of the cell.  
 2. The parameter `%%cython -a` will turn on verbose mode which is helpful when trying to debug why our function is slow.  This will show the resulting C program that is generated from our cython code.  Lines that are highlighted in <span style='background-color:yellow'>yellow</span> indicate that they are invoking a lot of python internals, and thus will be *generally slower*
 3. Adding types to parameters can help reduce the overhead of calling a function.  Try both of these expressions in a `%%cython -a` block
 ```cython
%cython -a
 def multiply_by_2(a):
      return a * 2
 cdef float multiply_by_2(float a):
      return a * 2
 ```
 4. Defining types on local variables can also help, some types you could use include: `dict`, `int`, `float`.  Defining a range parameter as `int` will help cython generate a C style for loop.  Using `dict` lets cython omit some extra checks when accessing the elements inside of a dictionary type
 ```cython
%cython -a
 cdef foo():
      cdef int i
      cdef dict d = {}
      for i  in range(10):
           d[i] = i
 ```
 5. We can replace `np.exp` with `libc.math.exp` which will use the `exp` function defined in C's `math.h` instead of making a slower call to numpy.
 

There is no *correct* answer to how fast your program should run.  You need to decide when your program is fast enough such that you can actually study the problems that you are interested in.  For this problem set, you should aim to get at least 2-3x faster that your baseline BLEU implementation.

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

from libc.math cimport exp, log
import numpy as np
cimport numpy as np

### STUDENTS START
### Helper functions for bleu
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

def faster_bleu(tuple yy, tuple aa):
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    

Check that our new BLEU function is predicting the same as our old function.  Also define a few additional checks of your own.

In [ ]:
assert bleu(yy=tuple('tEst'), aa=tuple('test')) == faster_bleu(yy=tuple('tEst'), aa=tuple('test'))

### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


Check how much faster the fast bleu function is compared to the baseline bleu function

  1. Baseline BLEU function runtime: <b><span style='color:red'>FILL IN</span> seconds</b>
  2. Faster BLEU function runtime: <b><span style='color:red'>FILL IN</span> seconds</b>
  3. Performance improvement (faster/baseline): <b><span style='color:red'>FILL IN</span> %</b>

In [ ]:
class FasterBleuStressTask(StressTask):
    
    def reward(self, *, aa, xx, yy):
        return faster_bleu(aa=aa, yy=yy)
    
faster_bleu_task = FasterBleuStressTask()

faster_bleu_max_bayes_value = MaximumBayesValueAgent(faster_bleu_task, trainable_linear_model)
%time faster_bleu_max_bayes_value.test(iterate_data('dev', max_examples=100))

Now that that our BLEU function is much faster, we can run it over the entire dataset:

In [ ]:
faster_bleu_max_bayes_value.test(iterate_data('dev'))

## Sampling from $p(\mathbf{y} \mid \mathbf{x})$

So far, our decision method has only been returning the *best* sample, and we have accomplished this by enumerating the entire domain of $\mathcal{Y}$.  However, it is often intractable to enumerate $\mathcal{Y}$, so we will instead sample a good $\mathbf y$.  

First we are just going to enumerate our entire domain of $\mathbf{Y}$ and compute the probability of all values $y$ as a baseline brute force sampling method.  You might find [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html) useful to draw a sample from this distribution.

In [ ]:
class BruteForceSampler(TrainableLinearStressModel):
    
    def sample(self, *, xx, oo=None):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END

brute_force_sampler = BruteForceSampler(task)
brute_force_sampler.initialize_params()

Check your sampler.  You should see some variations in predicted outputs.

In [ ]:
xx, oo, yy = next(iterate_data())
for i in range(20): print(brute_force_sampler.sample(xx=xx, oo=oo))

Enumerating the entire domain of $\mathcal{Y}$ can be slow, so now we are going to replace our brute force sampler with a Gibbs sampler.

The pseudo code for a Gibbs sampler looks like this:
  * $\mathbf{y}$ = initial start string
  * Repeat for $\mathbf{n}$ iterations:
    * Randomly choose a position $j$ such that (? in $\mathbf{o}$)
      * $y[j]$ = capital
      * $p_{\text{capital}} = \tilde{P}(y | x, o)$
      * $y[l]$ = lower case
      * $p_\text{lower} = \tilde{P}(y | x, o)$
      * make y[l] capital with probability $\frac{p_\text{capital}}{p_\text{lower} + p_\text{capital}}$
  * return y

In [ ]:
class GibbsSampler(TrainableLinearStressModel):
    
    def sample(self, *, xx, oo=None):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END

        
gibbs_sampler = GibbsSampler(task)
gibbs_sampler.initialize_params()

Check the Gibbs sampler.

In [ ]:
xx, oo, yy = next(iterate_data())
for i in range(20): print(gibbs_sampler.sample(xx=xx, oo=oo))

To check that our two sampling methods are generating samples from the same distribution, we are going to compute the Kullback-Leibler divergence.  The KL divergence computes the difference in the expected probability between our two distributions $P$ and $Q$.  Observe that the expression $\log \frac{P(y)}{Q(y)}$ will be zero in the case that both $P$ and $Q$ give the same probability to a value $\mathbf y$.

$$\text{KL}(P \mid\mid Q) = \sum_{\mathbf{y} \in \mathbf{Y}} P(\mathbf y) \log \frac{P(\mathbf y)}{Q(\mathbf y)}$$

We will approximate $P(\mathbf y)$ and $Q(\mathbf y)$ by generating 5000 samples from our two sampling methods and counting the number of times that $\mathbf y$ was generated.

You should get a value $< .1$ if both of your sampling methods are working correctly

In [ ]:
def compute_kl(method_1, method_2):
    xx, oo, yy = next(iterate_data())
    n = 5000
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
compute_kl(brute_force_sampler, gibbs_sampler)

Lets use our samplers to generate a few samples

In [ ]:
class SamplingAgent(DecisionAgent):
    
    def decision(self, *, xx, oo=None):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
agent_brute = SamplingAgent(task, brute_force_sampler)
agent_gibbs = SamplingAgent(task, gibbs_sampler)

First using the untrained linear classifier for our tasks

In [ ]:
agent_brute.test(iterate_data('dev'))

In [ ]:
agent_gibbs.test(iterate_data('dev'))

Now train and retest our samplers

In [ ]:
# train the classifier of our samplers
# for i in range(2):
#     brute_force_sampler.train_epoch(iterate_data())
# for i in range(2):
#     gibbs_sampler.train_epoch(iterate_data())

# Question: why can we replace the above lines and avoid retraining by
# copying the parameters from our already trained model
brute_force_sampler.params = trainable_linear_model.params
gibbs_sampler.params = trainable_linear_model.params

In [ ]:
agent_brute.test(iterate_data('dev'))

In [ ]:
agent_gibbs.test(iterate_data('dev'))

# The final test!

Throughout this homework we have implemented a number of different models, and we have been tuning, debugging and developing these models using our *development* set.  Now that we have reached the end, choose **one** of your models, decision rules, and tasks to run on the testing data and report this number.

Performance on the testing dataset: <span style='color:red'>FILL IN</span>

In [ ]:
### STUDENTS START
### my_chosen_task = ????
### my_chosen_model = ????
### my_chosen_agent = ????
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

my_chosen_agent.test(iterate_data('test'))